In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.exceptions import DataConversionWarning
import warnings
import pandas as pd
from pywt import wavedec
from pywt import waverec
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import VotingClassifier
import pywt
import scipy
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SVMSMOTE
import pylab as pl
from sklearn.utils import resample
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Input, Add, Flatten, Concatenate, MaxPool1D, Conv1D, Bidirectional, LSTM, Reshape

In [2]:
eeg1 = pd.read_csv("train_eeg1.csv").iloc[:, 1:]
eeg2 = pd.read_csv("train_eeg2.csv").iloc[:, 1:]
emg = pd.read_csv("train_emg.csv").iloc[:, 1:]
df_y = pd.read_csv("train_labels.csv").iloc[:, 1:]

## Balancing by sampling with replacement 

The idea is to pool all the under-represented REM 4-second signals together, then construct new examples as follows:

1) For the 1st sample, randomly select a signal and choose its 1st sample as the newly constructed signal's 1st sample

...

512) For the 512th sample, randomly select a signal and choose its 512th sample

In [7]:
eeg1np = eeg1.values
eeg2np = eeg2.values
emgnp = emg.values
ynp = df_y.values

In [13]:
#print(len(eeg2np[0])) 512
#rem_indices = np.where(ynp == 3)[0]
#print(len(rem_indices)) 3553
#print(len(ynp))  64800

512
3553
64800


In [17]:
eeg1_aug = []
eeg2_aug = []
emg_aug  = []

# approximately 6 percent of the total number of epochs, 64800, are REM in order to make them 
def samp_with_replacement(nbr_samples, class_lab):
    class_idxs = np.where(ynp == class_lab)[0]
    eeg1_f = eeg1np[class_idxs]
    eeg2_f = eeg2np[class_idxs]
    emg_f  = emgnp[class_idxs]
    
    # get the columns so we sample from them
    eeg1np_t = eeg1_f.T   # has 512 rows
    eeg2np_t = eeg2_f.T  # has 512 rows
    emgnp_t = emg_f.T   # has 512 rows
    
    samples_eeg1 = np.array([[0]*512]*nbr_samples)
    samples_eeg2 = np.array([[0]*512]*nbr_samples)
    samples_emg =  np.array([[0]*512]*nbr_samples)
    
    for i in range(512):
        samples_eeg1[:,i] = resample(eeg1np_t[i], replace=True, n_samples=nbr_samples, random_state=1)
        samples_eeg2[:,i] = resample(eeg2np_t[i], replace=True, n_samples=nbr_samples, random_state=2)
        samples_emg[:, i]  = resample(emgnp_t[i], replace=True,  n_samples=nbr_samples, random_state=3)
    
    global eeg1_aug, eeg2_aug, emg_aug
    eeg1_aug = np.vstack((eeg1np, samples_eeg1))
    eeg2_aug = np.vstack((eeg2np, samples_eeg2))
    emg_aug  = np.vstack((emgnp, samples_emg))
        

In [18]:
samp_with_replacement(10000, 3)

In [19]:
#print(len(eeg1_aug)) 74800
#print(len(eeg1np))  64800

74800
64800


#### Under-sample NREM and Wake classes
According to https://arxiv.org/pdf/1809.08443.pdf, the rebalancing was done such that REM was roughly 25% (instead of 5% originally), NREM was 33% and finally 42% from Wake. This leads to the equations

1) 13,553/(74,800 - x - y) = 0.25

2) (27,133-y)/(74,800 - x - y) = 0.33

3) (34,114-x)/(74,800 - x - y) = 0.42 (redundant)

which leads to x = 11345 (i.e. reduce Wake by 11345 signals) and y = 9,243 which means Wake has now 22,769 signals, and NREM has 17,890 signals out of 54,212 samples

In [30]:
y_aug = np.append(ynp, np.array([3]*10000))

In [32]:
under = RandomUnderSampler(sampling_strategy={1: 22769, 2:18890})   # technically this would give 34% for class NREM
# stack them horizontally then unstack later by slicing
X_new = np.hstack((eeg1_aug, eeg2_aug, emg_aug))
print(X_new.shape)
X_balanced, y_balanced = under.fit_resample(X_new, y_aug) 

(74800, 1536)


### Create 20 seconds segments by combining 5 of the 4-second length segments

In [35]:
## split them again
X_balanced = X_balanced[:-2, :]
print(len(X_balanced)) # to make sure length is multiple of 5
eeg1_bal = X_balanced[:, 0:512] 
eeg2_bal = X_balanced[:, 512:1024]
emg_bal  = X_balanced[:, 1024:1536]

55210


In [37]:
eeg1_reshaped = np.array([[0]*2560]*11042)
eeg2_reshaped = np.array([[0]*2560]*11042)
emg_reshaped = np.array([[0]*2560]*11042)
for m in range(11042):  # 11,042 = 55210/5 minus since end will do m + 1 (may cause out of bounds exc.)
    start = m*5
    end = m*5 + 5
    combined1 = ((eeg1_bal[start:end, :]).reshape(1, 2560))[0]
    combined2 = ((eeg2_bal[start:end, :]).reshape(1, 2560))[0]
    combined3 = ((emg_bal[start:end, :]).reshape(1, 2560))[0]
    eeg1_reshaped[m] = combined1
    eeg2_reshaped[m] = combined2
    emg_reshaped[m] = combined3

## RMS filter of the EMG signal

In [39]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = 1.0*np.ones(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))/float(window_size)

In [103]:
#rms_vals = np.array([[0]*20]*11042)    # window size is 1 second = 128 samples in each window
print(emg_reshaped.shape)
def rms_transform(row):
    res = np.array([])
    end = len(row)
    for i in range(end - 128):
        window = row[i:(i + 128)]
        temp = window_rms(window, 128)
        res = np.append(res, temp)
    return res
rms_vals= np.apply_along_axis(rms_transform, axis = 1, arr = emg_reshaped)

(11042, 2560)


In [104]:
print(rms_vals.shape)

(11042, 2432)


In [50]:
# this cell is not needed anymore
# for some reason there is a third dimension, to remove it:
#rms_vals = rms_vals[:, :, 0]
#print(rms_vals.shape)

(11042, 20)


## Combine EEG channels
so the rows of a matrix are actually two arrays corresponding to EEG1 and EEG2

In [60]:
eegs_comb = np.hstack((eeg1_reshaped, eeg2_reshaped))
print(eegs_comb.shape)
def combine_channels(signals):
    sigs = np.split(signals, 2)
    eeg1 = sigs[0]
    eeg2 = sigs[1]
    return np.array((eeg1, eeg2)).T
    
channels = np.apply_along_axis(combine_channels, axis = 1, arr = eegs_comb)
print(channels.shape)

(11042, 5120)
(11042, 2560, 2)


## CNN

### Feature Extraction - Model left branch

In [22]:
def left_branch_model():
    signal = Input(shape = (2560, 2))
    x = Conv1D(filters = 64, kernel_size = 50,  strides = 6, activation='relu')(signal)
    x = MaxPool1D(pool_size=8, strides=8)(x)
    x = Dropout(0.5)(x)
    x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu', padding = 'same')(x)
    x = MaxPool1D(pool_size=4, strides=4)(x)
    print(x.shape)
    return tf.keras.Model(inputs = signal, outputs = x)
lbranch = left_branch_model()

(None, 13, 128)


### Feature Extraction - Model middle branch

In [23]:
def mid_branch_model():
    signal = Input(shape = (2560, 2))
    x = Conv1D(filters = 64, kernel_size = 500,  strides = 50, activation='relu', padding = 'same')(signal) # try with and without padding = 'same'
    x = MaxPool1D(pool_size=8, strides=8)(x)
    x = Dropout(0.5)(x)
    x = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(x)
    x = MaxPool1D(pool_size=2, strides=2)(x)
    print(x.shape)
    return tf.keras.Model(inputs = signal, outputs = x)
mbranch = mid_branch_model()

(None, 3, 128)


### Feature Extraction - Model right branch

In [24]:
def right_branch_model():
    signal = Input(shape=(2432, 1))
    z = Conv1D(filters = 64, kernel_size = 500,  strides = 50, activation='relu', padding = 'same')(signal)
    z = MaxPool1D(pool_size=4, strides=4)(z)
    z = Dropout(0.5)(z)
    z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(z)
    z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(z)
    z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(z)
    z = MaxPool1D(pool_size=2, strides=2)(z)
    print(z.shape)
    return tf.keras.Model(inputs = signal, outputs = z)
rbranch = right_branch_model()

(None, 6, 128)


#### Concatenate along the second dimension (of sizes 13, 3, 6 respectively)

In [28]:
concat_layer = Concatenate(axis = 1)([lbranch.output, mbranch.output, rbranch.output])
concat_model = tf.keras.Model(inputs = [lbranch.input, mbranch.input, rbranch.input], outputs = concat_layer)

### Scoring

In [47]:
s = Dropout(0.5)(inputs = concat_model.output)

# Left branch (cannot flatten before since LSTM requires input of form (None, x, y) NOT (None, z))
sLeft = Bidirectional(LSTM(units = 512, return_sequences = True, activation='tanh'))(s) #https://stackoverflow.com/questions/40331510/how-to-stack-multiple-lstm-in-keras
sLeft = Bidirectional(LSTM(units = 512, activation='tanh'))(sLeft)
print(sLeft.shape)

# Right branch
q = Flatten()(s)
q = Dense(units = 1024, activation = 'relu')(q)
print(q.shape)

# combine both 1024-length vectors by adding them
q = Add()([q, sLeft])
print(q.shape)

# now dropout at 

(None, 1024)
(None, 1024)
(None, 1024)


In [190]:
#Left most CNN
signal1 = Input(shape=(2560, 2))
x = Conv1D(filters = 64, kernel_size = 50,  strides = 6, activation='relu')(signal1)
x = MaxPool1D(pool_size=8, strides=8)(x)
x = Dropout(0.5)(x)
x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu')(x)
x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu')(x)
x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu')(x)
x = MaxPool1D(pool_size=4, strides=4)(x)
x = Reshape((14, 64))(x)

#Middle most CNN
signal2 = Input(shape=(2560, 2))
y = Conv1D(filters = 64, kernel_size = 500, strides = 50, activation='relu')(signal2)
y = MaxPool1D(pool_size=4, strides=4)(y)
y = Dropout(0.5)(y)
print(y.shape)
y = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu')(y)
print(y.shape)
y = Reshape((10, 64))(y)
y = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu')(y)
print(y.shape)
y = Reshape((10, 64))(y)
y = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu')(y)
y = MaxPool1D(pool_size=2, strides=2)(y)
y = Reshape((4, 64))(y)

#Right CNN
signal_emg = Input(shape=(2432, 1))
print(signal_emg.shape)
z = Conv1D(filters = 64, kernel_size = 500,  strides = 50, activation='relu')(signal_emg)
print(z.shape)
z = MaxPool1D(pool_size=4, strides=4)(z)
print(z.shape)
z = Dropout(0.5)(z)
print(z.shape)
z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'valid')(z)
print(z.shape)
z = Reshape((8, 64))(z)
#print('**************')
print(z.shape)
z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'valid')(z)
print(z.shape)
z = Reshape((6, 64))(z)
print(z.shape)
z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'valid')(z)
print(z.shape)
z = Reshape((2, 64))(z)
z = MaxPool1D(pool_size=2, strides=2)(z)
print(z.shape)

prelim_model = Concatenate(axis = 1)([x, y, z])  # NOT sure at all if I should flatten before concatenation instead of doing axis = 1
p2 = Dropout(0.5)(prelim_model)
print(p2.shape)
p3 = Bidirectional(LSTM(units = 512, activation='tanh'))(p2)
print(p3.shape)
p3 = Reshape((16, 64))(p3)
p4 = Bidirectional(LSTM(units = 512, activation='tanh'))(p3)

q2 = Flatten()(p2)
q3 = Dense(units = 1024, activation = 'relu')(q2)
q3 = Flatten()(q3)
q4 = Flatten()(p4)
print("q4 has shape "+ str(q4.shape))
print("q3 has shape "+ str(q3.shape))

out = Add()([q3, q4])
out = Dropout(0.5)(q5)
print(out.shape)
out = Dense(3, activation='softmax')(out)


(None, 10, 64)
(None, 5, 128)
(None, 5, 128)
(None, 2432, 1)
(None, 39, 64)
(None, 9, 64)
(None, 9, 64)
(None, 4, 128)
(None, 8, 64)
(None, 3, 128)
(None, 6, 64)
(None, 1, 128)
(None, 1, 64)
(None, 19, 64)
(None, 1024)
q4 has shape (None, 1024)
q3 has shape (None, 1024)
(None, 3)


In [ ]:
model = Model([signal1, signal2, signal_emg], out)
print(model.summary())
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
X = [np.zeros((1,27,27,1))] * 2
y = np.ones((1,1))
model.fit(X, y)

In [53]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [61]:
print(eeg1.head())

         x1        x2        x3       x4        x5       x6        x7  \
0  0.000400  0.000470  0.000067 -0.00016 -0.000003  0.00031  0.000360   
1  0.000067  0.000095  0.000270  0.00028  0.000250  0.00012  0.000094   
2  0.000160 -0.000210 -0.000840 -0.00120 -0.001200 -0.00140 -0.001400   
3 -0.000140  0.000260  0.000390  0.00043  0.000280  0.00023  0.000390   
4 -0.001100 -0.000790 -0.000081  0.00014  0.000200 -0.00014 -0.000430   

        x8        x9      x10  ...      x503      x504      x505      x506  \
0  0.00019 -0.000072 -0.00007  ... -0.000086  0.000033 -0.000046 -0.000270   
1 -0.00034 -0.000960 -0.00120  ...  0.000046  0.000300  0.000630  0.000710   
2 -0.00091 -0.000600 -0.00027  ... -0.000680 -0.000880 -0.001000 -0.000770   
3  0.00022  0.000150  0.00022  ...  0.000720  0.000760  0.000380  0.000052   
4 -0.00053 -0.000580 -0.00041  ...  0.000290  0.000600  0.000670  0.000190   

       x507     x508     x509     x510      x511      x512  
0 -0.000390 -0.00034 -0.00032 -